# 一、载入数据

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats, integrate

%matplotlib inline

In [2]:
data_path = '../data/'
df_r = pd.read_csv(data_path + 'train_rice.csv', encoding='gb2312')
df_w = pd.read_csv(
    data_path + 'train_weather.csv', encoding='gb2312', low_memory=False)

In [5]:
df_w.head()

,区县id,站名id,年份,月份,日期,日照时数（单位：h),02时风向,08时风向,14时风向,20时风向,日平均风速(单位：m/s),日降水量（mm）,日最高温度（单位：℃）,日最低温度（单位：℃）,日平均温度（单位：℃）,日相对湿度（单位：%）,日平均气压（单位：hPa）
0,county25,1,2015,1,1,6.3,NNW,N,NNE,S,1.8,0,14.1,7.2,9.1,43,980.6
1,county25,1,2015,1,2,8.1,SSE,S,SSW,S,1.3,0,18.3,1.6,7.5,64,977.7
2,county25,1,2015,1,3,4.3,S,SSE,S,SSE,2,0,18.9,2.4,9.4,66,973.1
3,county25,1,2015,1,4,0,WNW,SSW,S,SSE,2.2,0.3,14.9,9.0,11.5,76,966.7
4,county25,1,2015,1,5,0,SSE,S,SSW,C,2.8,0.8,16.8,11.6,15.5,91,963.9


In [6]:
df_r.head()

,区县id,2015年早稻,2016年早稻,2017年早稻,2015年晚稻,2016年晚稻,2017年晚稻
0,county1,15.812589,15.878158,15.835681,15.754182,15.740001,15.636155
1,county2,16.658008,16.623931,16.555916,16.666867,16.686079,16.583385
2,county3,15.206326,15.085393,15.049788,15.177311,14.995996,14.889857
3,county4,15.352061,15.311764,15.236604,15.020413,15.005488,14.882306
4,county5,15.942749,15.943796,15.876010,16.019721,15.989249,15.905980


# 二、数据分析

## 1.字符串转化为浮点数

In [7]:
def rpf(x):
    try:
        t = float(x)
    except:
        t = np.nan
    return t

In [8]:
dc = [
    '日照时数（单位：h)', '日平均风速(单位：m/s)', '日降水量（mm）', '日最高温度（单位：℃）', '日最低温度（单位：℃）',
    '日平均温度（单位：℃）', '日相对湿度（单位：%）', '日平均气压（单位：hPa）'
]

In [9]:
for col in dc:
    df_w[col] = df_w[col].apply(rpf)

In [10]:
df_w.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132951 entries, 0 to 132950
Data columns (total 17 columns):
区县id             132951 non-null object
站名id             132951 non-null int64
年份               132951 non-null int64
月份               132951 non-null int64
日期               132951 non-null int64
日照时数（单位：h)       132932 non-null float64
02时风向            132951 non-null object
08时风向            132951 non-null object
14时风向            132951 non-null object
20时风向            132951 non-null object
日平均风速(单位：m/s)    132938 non-null float64
日降水量（mm）         121132 non-null float64
日最高温度（单位：℃）      132951 non-null float64
日最低温度（单位：℃）      132951 non-null float64
日平均温度（单位：℃）      132944 non-null float64
日相对湿度（单位：%）      132945 non-null float64
日平均气压（单位：hPa）    132929 non-null float64
dtypes: float64(8), int64(4), object(5)
memory usage: 17.2+ MB


## 2、站点数据

In [11]:
df_w['站名id'].value_counts()

1    128568
2      4383
Name: 站名id, dtype: int64

In [12]:
df_w.loc[df_w['站名id'] == 2]['区县id'].value_counts()

county8     1461
county1     1461
county34    1461
Name: 区县id, dtype: int64

county1、county8、county34 有站点1和站点2，其余 county只有站点1

## 3.数据缺失率

In [13]:
def get_miss(df, columns):
    miss = []
    for c in columns:
        miss.append([c, round(df[c].isnull().mean() * 100, 2)])
    return sorted(miss, key=lambda x: x[1], reverse=True)

In [14]:
# 缺失率（%）
miss = get_miss(df_w, dc)
miss

[['日降水量（mm）', 8.89],
 ['日平均气压（单位：hPa）', 0.02],
 ['日照时数（单位：h)', 0.01],
 ['日平均风速(单位：m/s)', 0.01],
 ['日平均温度（单位：℃）', 0.01],
 ['日最高温度（单位：℃）', 0.0],
 ['日最低温度（单位：℃）', 0.0],
 ['日相对湿度（单位：%）', 0.0]]

## 4.月数据统计

In [15]:
df_ws = df_w.groupby(['区县id', '站名id', '年份', '月份'])[dc].sum()

In [31]:
df_ws.head(24)

日照时数（单位：h)  日平均风速(单位：m/s)  日降水量（mm）  日最高温度（单位：℃）  \
区县id    站名id 年份   月份                                                     
county1 1    2015 1        108.8           46.2      37.4        577.6   
                  2         77.7           37.9      22.5        573.9   
                  3         45.8           46.4      42.3        677.6   
                  4        155.1           50.1      43.5        856.9   
                  5        154.4           55.0      99.0        993.0   
                  6        182.2           63.4      79.8       1010.3   
                  7        141.7           54.4     235.3        994.9   
                  8        182.9           46.8     242.8       1017.5   
                  9        113.5           46.1     131.8        920.1   
                  10       169.3           42.1      56.6        895.2   
                  11        61.9           35.5      98.1        735.7   
                  12        37.8           41.2     133.2        528.8   
             2016 1         37.3           43.4     129.1        488.2   
                  2         95.8           47.0      19.0        521.3   
                  3         31.7           43.2      38.6        657.8   
                  4         91.9           54.6     130.8        871.3   
                  5        154.4           58.4     203.8        973.0   
                  6        200.6           52.6     345.2       1016.4   
                  7        191.6           57.0      76.0       1051.4   
                  8        166.5           47.5     355.2       1039.0   
                  9        179.6           45.4      47.2        970.7   
                  10       187.5           54.0     152.8        940.3   
                  11       101.8           51.8      44.6        719.2   
                  12       144.2           49.2       4.1        684.2   

                      日最低温度（单位：℃）  日平均温度（单位：℃）  日相对湿度（单位：%）  日平均气压（单位：hPa）  
区县id    站名id 年份   月份                                                        
county1 1    2015 1         312.4        419.8       2578.0        31200.6  
                  2         375.0        458.8       2367.0        28094.0  
                  3         487.9        567.9       2705.0        31043.1  
                  4         544.6        682.7       2310.0        29945.7  
                  5         747.3        846.6       2557.0        30744.6  
                  6         765.3        870.8       2402.0        29710.0  
                  7         754.0        855.1       2513.0        30671.7  
                  8         761.8        867.3       2535.0        30746.5  
                  9         706.7        787.7       2600.0        29868.1  
                  10        593.2        713.4       2561.0        31017.8  
                  11        536.5        615.2       2586.0        30088.2  
                  12        369.5        434.8       2623.0        31227.2  
             2016 1         331.5        392.1       2678.0        31188.5  
                  2         242.3        357.3       2214.0        29235.3  
                  3         454.7        545.2       2600.0        31058.3  
                  4         639.8        735.2       2432.0        29837.9  
                  5         707.2        822.9       2467.0        30800.2  
                  6         765.4        865.9       2405.0        29748.5  
                  7         795.6        898.9       2480.0        30726.1  
                  8         781.5        878.5       2560.0        30645.9  
                  9         705.5        812.3       2370.0        29818.0  
                  10        653.1        779.0       2303.0        30914.8  
                  11        487.6        580.7       2443.0        30097.8  
                  12        373.0        500.6       2369.0        31171.8

## 5.均值统计

In [18]:
df_r['早稻均值'] = df_r[['2015年早稻', '2016年早稻', '2017年早稻']].mean(axis=1).values
df_r['晚稻均值'] = df_r[['2015年晚稻', '2016年晚稻', '2017年晚稻']].mean(axis=1).values
df_r = df_r[[
    '区县id', '2015年早稻', '2016年早稻', '2017年早稻', '早稻均值', '2015年晚稻', '2016年晚稻',
    '2017年晚稻', '晚稻均值'
]]

In [21]:
df_r.head(10)

,区县id,2015年早稻,2016年早稻,2017年早稻,早稻均值,2015年晚稻,2016年晚稻,2017年晚稻,晚稻均值
0,county1,15.812589,15.878158,15.835681,15.842143,15.754182,15.740001,15.636155,15.710113
1,county2,16.658008,16.623931,16.555916,16.612618,16.666867,16.686079,16.583385,16.645444
2,county3,15.206326,15.085393,15.049788,15.113835,15.177311,14.995996,14.889857,15.021054
3,county4,15.352061,15.311764,15.236604,15.300143,15.020413,15.005488,14.882306,14.969402
4,county5,15.942749,15.943796,15.876010,15.920852,16.019721,15.989249,15.905980,15.971650
5,county6,17.306096,17.346304,17.319839,17.324080,17.099665,17.174718,17.066806,17.113730
6,county7,17.305326,17.285705,17.209291,17.266774,17.323740,17.331463,17.292745,17.315983
7,county8,12.829017,12.931724,12.724834,12.828525,12.318557,12.437541,12.422412,12.392837
8,county9,15.979893,16.032551,15.923424,15.978623,15.930654,15.938682,15.869497,15.912944
9,county10,16.240918,16.294254,16.231479,16.255550,15.861623,15.955306,15.817218,15.878049


## 6.构造Features与Labels

In [22]:
countyId = df_r['区县id'].values

In [23]:
# df: 特征， df_l: 标签， postfix: 结果，state： 站点
def makeTrainData(df, df_l, postfix, countyId, years, months, state=1):
    res = pd.DataFrame()
    for countyid in countyId:
        for year in years:
            label = 0
            if year != 2018:
                label = df_l.loc[df_l['区县id'] == countyid][str(year) +
                                                           postfix].values[0]
            features = [label]
            for m in months:
                features += df.loc[(countyid, state, year, m)].values.tolist()
            res[countyid + '_' + str(year)] = features
    return res

In [34]:
df_ws.head()

日照时数（单位：h)  日平均风速(单位：m/s)  日降水量（mm）  日最高温度（单位：℃）  \
区县id    站名id 年份   月份                                                     
county1 1    2015 1        108.8           46.2      37.4        577.6   
                  2         77.7           37.9      22.5        573.9   
                  3         45.8           46.4      42.3        677.6   
                  4        155.1           50.1      43.5        856.9   
                  5        154.4           55.0      99.0        993.0   

                      日最低温度（单位：℃）  日平均温度（单位：℃）  日相对湿度（单位：%）  日平均气压（单位：hPa）  
区县id    站名id 年份   月份                                                        
county1 1    2015 1         312.4        419.8       2578.0        31200.6  
                  2         375.0        458.8       2367.0        28094.0  
                  3         487.9        567.9       2705.0        31043.1  
                  4         544.6        682.7       2310.0        29945.7  
                  5         747.3        846.6       2557.0        30744.6

### 6.1 早稻

In [24]:
trainData_1 = makeTrainData(
    df_ws,
    df_r,
    postfix='年早稻',
    countyId=countyId,
    years=[2015, 2016, 2017, 2018],
    months=[3, 4, 5, 6, 7, 8])
trainData_1.head()

,county1_2015,county1_2016,county1_2017,county1_2018,county2_2015,county2_2016,county2_2017,county2_2018,county3_2015,county3_2016,...,county86_2017,county86_2018,county87_2015,county87_2016,county87_2017,county87_2018,county88_2015,county88_2016,county88_2017,county88_2018
0,15.812589,15.878158,15.835681,0.0,16.658008,16.623931,16.555916,0.0,15.206326,15.085393,...,14.984693,0.0,14.767403,14.807953,14.773236,0.0,12.69773,13.127092,12.72598,0.0
1,45.800000,31.700000,40.300000,120.3,27.800000,20.500000,33.400000,90.6,62.400000,57.200000,...,38.300000,104.0,43.000000,51.500000,50.000000,127.2,45.10000,34.200000,48.50000,121.4
2,46.400000,43.200000,89.900000,83.1,29.700000,30.700000,71.000000,77.1,72.300000,76.000000,...,57.300000,55.6,62.800000,62.600000,62.800000,70.0,59.90000,59.200000,56.10000,60.4
3,42.300000,38.600000,164.900000,69.5,55.100000,27.700000,167.400000,68.2,26.900000,24.100000,...,120.200000,34.6,23.100000,20.900000,129.800000,66.0,63.10000,50.100000,145.60000,73.9
4,677.600000,657.800000,658.400000,756.3,690.700000,668.900000,649.200000,748.0,682.800000,658.200000,...,684.100000,770.6,653.400000,637.500000,649.100000,717.3,668.80000,649.800000,673.40000,769.9


In [33]:
trainData_1

,county1_2015,county1_2016,county1_2017,county1_2018,county2_2015,county2_2016,county2_2017,county2_2018,county3_2015,county3_2016,...,county86_2017,county86_2018,county87_2015,county87_2016,county87_2017,county87_2018,county88_2015,county88_2016,county88_2017,county88_2018
0,15.812589,15.878158,15.835681,0.0,16.658008,16.623931,16.555916,0.0,15.206326,15.085393,...,14.984693,0.0,14.767403,14.807953,14.773236,0.0,12.69773,13.127092,12.72598,0.0
1,45.800000,31.700000,40.300000,120.3,27.800000,20.500000,33.400000,90.6,62.400000,57.200000,...,38.300000,104.0,43.000000,51.500000,50.000000,127.2,45.10000,34.200000,48.50000,121.4
2,46.400000,43.200000,89.900000,83.1,29.700000,30.700000,71.000000,77.1,72.300000,76.000000,...,57.300000,55.6,62.800000,62.600000,62.800000,70.0,59.90000,59.200000,56.10000,60.4
3,42.300000,38.600000,164.900000,69.5,55.100000,27.700000,167.400000,68.2,26.900000,24.100000,...,120.200000,34.6,23.100000,20.900000,129.800000,66.0,63.10000,50.100000,145.60000,73.9
4,677.600000,657.800000,658.400000,756.3,690.700000,668.900000,649.200000,748.0,682.800000,658.200000,...,684.100000,770.6,653.400000,637.500000,649.100000,717.3,668.80000,649.800000,673.40000,769.9
5,487.900000,454.700000,458.400000,512.2,524.500000,489.500000,461.300000,523.5,517.700000,475.600000,...,493.100000,504.5,500.100000,461.400000,467.700000,506.9,505.60000,472.100000,487.60000,522.8
6,567.900000,545.200000,531.600000,613.7,588.400000,564.500000,531.600000,616.0,577.900000,552.900000,...,565.000000,617.3,561.500000,532.200000,534.400000,599.0,566.00000,541.100000,557.80000,620.9
7,2705.000000,2600.000000,2643.000000,2452.0,2651.000000,2424.000000,2618.000000,2518.0,2618.000000,2536.000000,...,2554.000000,2466.0,2602.000000,2549.000000,2658.000000,2432.0,2781.00000,2693.000000,2674.00000,2458.0
8,31043.100000,31058.300000,30915.600000,30902.4,31085.200000,31088.700000,30902.200000,30890.6,31018.200000,31032.000000,...,30449.000000,30438.5,29845.800000,29859.200000,29830.200000,29821.6,30217.20000,30234.200000,30195.70000,30193.5
9,155.100000,91.900000,105.700000,125.6,155.100000,63.800000,79.800000,100.7,217.900000,113.600000,...,86.500000,86.6,159.500000,96.500000,90.900000,116.3,171.50000,97.600000,95.10000,85.3


### 6.2 晚稻

In [25]:
trainData_2 = makeTrainData(
    df_ws,
    df_r,
    postfix='年晚稻',
    countyId=countyId,
    years=[2015, 2016, 2017, 2018],
    months=[8, 9, 10])
trainData_2.head()

,county1_2015,county1_2016,county1_2017,county1_2018,county2_2015,county2_2016,county2_2017,county2_2018,county3_2015,county3_2016,...,county86_2017,county86_2018,county87_2015,county87_2016,county87_2017,county87_2018,county88_2015,county88_2016,county88_2017,county88_2018
0,15.754182,15.740001,15.636155,0.0,16.666867,16.686079,16.583385,0.0,15.177311,14.995996,...,15.125234,0.0,14.828394,14.817119,14.775778,0.0,12.419692,12.340206,11.806864,0.0
1,182.900000,166.500000,195.700000,171.9,155.300000,204.500000,183.100000,160.2,219.900000,233.200000,...,148.500000,147.3,162.400000,197.200000,175.000000,178.0,166.600000,158.500000,143.500000,124.8
2,46.800000,47.500000,74.300000,61.9,32.400000,34.700000,69.800000,56.1,64.400000,64.200000,...,46.500000,33.4,45.800000,46.400000,53.600000,52.0,52.500000,47.200000,53.400000,33.2
3,242.800000,355.200000,256.100000,151.8,82.200000,374.400000,297.900000,240.5,134.400000,303.200000,...,224.200000,316.6,99.000000,300.600000,245.100000,346.4,344.900000,274.500000,206.700000,509.7
4,1017.500000,1039.000000,1006.500000,1013.0,1064.800000,1066.800000,1006.400000,1022.3,1023.100000,1043.800000,...,985.400000,1004.3,960.500000,995.900000,946.700000,961.4,982.600000,1008.400000,959.600000,973.1
